In [ ]:
results_file = "data/eval/results.json"
score = "bleu1"
ignore = {
    "task": None,
    "strategy": ["regex"],
    "msid": None,
    "figure_label": None,
    "run": None,
}

In [ ]:
import pandas as pd

df = pd.read_json(results_file)

# remove runs in ignore
for k, v in ignore.items():
    if v is not None:
        df = df[~df[k].isin(v)]

# run is a number, but should be considered a category
df = df.assign(run=df.run.astype('category'))

n_tasks = df.task.nunique()
n_strategies = df.strategy.nunique()
n_msids = df.msid.nunique()
n_figures = df.figure_label.nunique()
n_runs = df.run.nunique()

assert df[score].min() >= 0, "Score should be non-negative"
assert df[score].max() <= 1, "Score should be at most 1"

# split into tasks
legends = df[df["task"] == "extract_figure_legends"]
labels = df[df["task"] == "extract_figures"]
titles = df[df["task"] == "extract_figure_title"]
captions = df[df["task"] == "extract_figure_caption"]

## Top-line stats

In [ ]:
df.groupby(["task", "strategy"])[[score]].describe()

## Graphs

In [ ]:
import plotly.express as px

def add_threshold(df, fig, axis="y"):
    score_threshold = df[f"{score}_threshold"].iloc[0].round(2)
    kwargs = dict(
        line_dash="dash",
        line_color="green",
        opacity=0.5,
        annotation_text=f"threshold: {score_threshold}",
        annotation_position="top left",
    )
    if axis == "y":
        fig.add_hline(y=score_threshold, **kwargs)
    else:
        fig.add_vline(x=score_threshold, **kwargs)

def hist_score(df, interval_width=0.01, **kwargs):
    groupby = ["task", "strategy"]
    x_start = 0 - interval_width
    x_end = 1 + interval_width
    y_start = -1
    y_end = df.groupby(groupby).count().max().max() + 1
    fig = px.histogram(
        df,
        x=score,
        range_x=[x_start, x_end],
        range_y=[y_start, y_end],
        facet_col="task",
        facet_row="strategy",
        width=900,
        height=100 + 250 * n_strategies,
        **kwargs,
    )
    fig.update_traces(
        xbins=dict( # bins used for histogram
            start=x_start,
            end=x_end,
            size=interval_width,
        )
    )
    fig.update_layout(bargap=0.2)
    add_threshold(df, fig, axis="x")
    return fig

def scatter_score(df, **kwargs):
    fig = px.scatter(
        df,
        x="msid",
        y=score,
        facet_col="task",
        facet_row="strategy",
        color="run",
        height=100 + 250 * n_strategies,
        width=900,
        **kwargs,
    )
    fig.update_traces(marker_size=10)
    fig.update_layout(scattermode="group", scattergap=0.9)
    add_threshold(df, fig)
    return fig

def plot_std(df, groupby, **kwargs):
    df_std = (
        df.groupby(groupby)
        [score].std()
        .reset_index()
        .sort_values(groupby)
        .rename(columns={score: f"{score}_std"})
    )
    fig = px.scatter(
        df_std,
        x="msid",
        y=f"{score}_std",
        facet_col="task",
        facet_row="strategy",
        height=100 + 250 * n_strategies,
        width=900,
        **kwargs,
    )
    return fig

In [ ]:
for task_df in [legends, labels]:
    if task_df.empty:
        continue
    hist_score(task_df).show()
    scatter_score(task_df).show()
    plot_std(task_df, groupby=["task", "strategy", "msid"]).show()

In [ ]:
for task_df in [titles, captions]:
    if task_df.empty:
        continue
    hist_score(task_df).show()
    scatter_score(
        task_df,
        symbol="figure_label",
    ).show()
    plot_std(
        task_df,
        groupby=["task", "strategy", "msid", "figure_label"],
        color="figure_label",
        symbol="figure_label",
    ).show()

## Run details

## Run details

In [ ]:
import difflib
import html
from IPython.display import display, HTML

def inline_diff(a, b):
    matcher = difflib.SequenceMatcher(None, a, b)
    def process_tag(tag, i1, i2, j1, j2):
        a_text = html.escape(a[i1:i2])
        b_text = html.escape(b[j1:j2])
        if tag == 'delete':
            return '<del>' + a_text + '</del>'
        if tag == 'equal':
            return a_text
        if tag == 'insert':
            return '<ins>' + b_text + '</ins>'
        if tag == 'replace':
            # combine as delete + insert
            return '<del>' + a_text + '</del><ins>' + b_text + '</ins>'
        assert False, "Unknown tag %r"%tag
    return ''.join(process_tag(*t) for t in matcher.get_opcodes())

def display_diff_(row):
    diff_css = """
    <style>
    ins {background-color: #588a41;}  // light green
    del {background-color: #ffaaaa;}  // light red
    repl {background-color: #bb99ff;} // light purple
    table tr > * { width: 50%; }
    table tr > td { vertical-align: top; text-align: left; }
    </style>
    """

    # run_name = (
    #     f"{row.task} - {row.strategy} - {row.msid} - {row.run}"
    #     if row.task == "extract_figure_caption"
    #     else f"{row.task} - {row.strategy} - {row.msid} - {row.figure_label} - {row.run}"
    # )
    # header = f"<h3>{run_name}</h3>"
    section_fn = lambda title, content: f"<h4>{title}</h4><p>{content}</p>"
    # scores = section_fn("Scores", f"bleu1: {row.bleu1[0]:.2f}}")

    comparison = f"""
        <table>
            <thead>
                <tr>
                    <th>Expected</th>
                    <th>Actual</th>
                </tr>
            </thead>
            <tbody>
                <tr>
                    <td>{html.escape(row.head(1)["expected"].values[0])}</td>
                    <td>{html.escape(row.head(1)["actual"].values[0])}</td>
                </tr>
            </tbody>
        </table>
    """
    diff = section_fn("Diff", inline_diff(row.head(1)["expected"].values[0], row.head(1)["actual"].values[0]) + comparison)
    input_text = section_fn("Input", row.input)

    # display(HTML(diff_css + header + scores + diff + input_text))
    display(HTML(diff_css + diff + input_text))


In [ ]:
df["task"].unique()

array(['extract_figure_legends', None, 'extract_figures',
       'extract_figure_title', 'extract_figure_caption'], dtype=object)

In [ ]:
df_bad_figure_legends = df[(df["task"]=="extract_figure_legends") & (df["bleu1"]<0.95)]
df_bad_labels = df[(df["task"]=="extract_figures") & (df["bleu1"]<0.95)]
df_bad_titles = df[(df["task"]=="extract_figure_title") & (df["bleu1"]<0.95)]
df_bad_caption = df[(df["task"]=="extract_figure_caption") & (df["bleu1"]<0.95)]


## Examples of bad detection of figure section

In [ ]:
display_diff_(df_bad_figure_legends.sample()) if len(df_bad_figure_legends) > 0 else print("No bad figure legends")

No bad figure legends


## Examples of bad detection of figure label

In [ ]:
display_diff_(df_bad_labels.sample()) if len(df_bad_labels) > 0 else print("No bad figure labels")

Expected,Actual
Figure 1 Figure 2 Figure 3 Figure 4 Figure 5 Figure 6 Figure 7 Figure 8,Figure 7 Figure 8


## Examples of bad figure titles

In [ ]:
display_diff_(df_bad_titles.sample()) if len(df_bad_titles) > 0 else print("No bad figure titles")

Expected,Actual
<strong>Recruitment of Nedd4L to membrane tubules generated by FCHO2 in cells.</strong>,Recruitment of Nedd4L to membrane tubules generated by FCHO2 in cells.


## Examples of bad figure captions

In [ ]:
display_diff_(df_bad_caption.sample()) if len(df_bad_caption) > 0 else print("No bad figure captions")

Expected,Actual
"(<strong>A)</strong> HEK239 cells transiently expressing the <sup>35</sup>S-NHK are chased for the indicated time without (lanes 1-3) or with PS341 (lanes 4, 5), PS341/BafA1 (lanes 6, 7), KIF (lanes 8, 9), or KIF/BafA1 (10-11). Radiolabeled NHK is immunoisolated at the end of the indicated chase time and separated in SDS-PAGE. Densitometric quantification of the <sup>35</sup>S-NHK band in this representative experiment is shown. (<strong>B)</strong> Quantification of <sup>35</sup>S-NHK after a 10- or 120-min chase in cells mock treated or exposed to PS341 or PS341/BafA1. Individual data points at 120-min (%, normalized to 10’ mock): mock= 49, 37 and 49%; PS341= 73, 55 and 52%; PS341/BafA1= 91, 63 and 68%. (<strong>C)</strong> Same as (<strong>B)</strong> for Kif, KIF/BafA1. Individual data points at 120-min (%, normalized to 10-min mock): mock= 49, 37 and 49%; KIF= 80, 53 and 54%; KIF/BafA1= 99, 64 and 83%. <strong>(D)</strong> BafA1 does not inhibit clearance of the ERAD client NHK. n=3 for 10- and 120-min; n=2 for 120-min BafA1. (<strong>E)</strong> ATZ, a canonical ERLAD client accumulates within LAMP1-positive endolysosomes in 3T3 cells exposed to BafA1. (<strong>F)</strong> The ERAD client NHK is not delivered within LAMP1-positive endolysosomes. (<strong>G-H)</strong> NHK is delivered and accumulates within endolysosomes inactivated with BafA1 upon ERAD inhibition with KIF (G) or with PS341 (H). (<strong>I)</strong> LysoQuant quantification of ATZ delivery within LAMP1-positive endolysosomes in panel <strong>E</strong>, and of NHK in panels <strong>F-H</strong> (n=18, 19, 23 and 13 cells, respectively. (<strong>J-N)</strong> Analysis of NHK delivery to LAMP1-positive endolysosomes as in (F-H) but with drug incubation of 8h and with additional BafA1 washout conditions (L, N).<strong>O</strong> LysoQuant quantification of NHK accumulation within LAMP1 positive endolysosomes (panels <strong>J-N</strong>, n=31, 24, 31, 31 and 33 cells respectively.</p> <p>Data Information: (B, C) N=3 independent experiments, mean +/- SEM is shown. (D) mean +/- SEM is shown.(I, O) mean is shown. N=3 independent experiments. One-way analysis of variance (ANOVA) and Dunnett’s multiple comparison test, ****P&lt;0.0001. (E-H, J-N) Scale bars: 10μM, Insets are shown with white squares.</p> <p><strong><br /> </strong></p>","Figure 2-The ERAD client NHK is re-directed to LAMP1-positive endolysosomes for degradation upon pharmacologic inactivation of ERAD. (A) HEK239 cells transiently expressing the 35S-NHK are chased for the indicated time without (lanes 1-3) or with PS341 (lanes 4, 5), PS341/BafA1 (lanes 6, 7), KIF (lanes 8, 9), or KIF/BafA1 (10-11). Radiolabeled NHK is immunoisolated at the end of the indicated chase time and separated in SDS-PAGE. Densitometric quantification of the 35S-NHK band in this representative experiment is shown. (B) Quantification of 35S-NHK after a 10- or 120-min chase in cells mock treated or exposed to PS341 or PS341/BafA1. Individual data points at 120-min (%, normalized to 10’ mock): mock= 49, 37 and 49%; PS341= 73, 55 and 52%; PS341/BafA1= 91, 63 and 68%. (C) Same as (B) for Kif, KIF/BafA1. Individual data points at 120-min (%, normalized to 10-min mock): mock= 49, 37 and 49%; KIF= 80, 53 and 54%; KIF/BafA1= 99, 64 and 83%. (D) BafA1 does not inhibit clearance of the ERAD client NHK. n=3 for 10- and 120-min; n=2 for 120-min BafA1. (E) ATZ, a canonical ERLAD client accumulates within LAMP1-positive endolysosomes in 3T3 cells exposed to BafA1. (F) The ERAD client NHK is not delivered within LAMP1-positive endolysosomes. (G-H) NHK is delivered and accumulates within endolysosomes inactivated with BafA1 upon ERAD inhibition with KIF (G) or with PS341 (H). (I) LysoQuant quantification of ATZ delivery within LAMP1-positive endolysosomes in panel E, and of NHK in panels F-H (n=18, 19, 23 and 13 cells, respectively. (J-N) Analysis of NHK delivery to LAMP1-positive endolysosomes as in (F-H) but with drug i

In [ ]:
# import difflib
# import html
# from IPython.display import display, HTML

# def inline_diff(a, b):
#     matcher = difflib.SequenceMatcher(None, a, b)
#     def process_tag(tag, i1, i2, j1, j2):
#         a_text = html.escape(a[i1:i2])
#         b_text = html.escape(b[j1:j2])
#         if tag == 'delete':
#             return '<del>' + a_text + '</del>'
#         if tag == 'equal':
#             return a_text
#         if tag == 'insert':
#             return '<ins>' + b_text + '</ins>'
#         if tag == 'replace':
#             # combine as delete + insert
#             return '<del>' + a_text + '</del><ins>' + b_text + '</ins>'
#         assert False, "Unknown tag %r"%tag
#     return ''.join(process_tag(*t) for t in matcher.get_opcodes())

# def display_diff(row):
#     diff_css = """
#     <style>
#     ins {background-color: #aaffaa;}  // light green
#     del {background-color: #ffaaaa;}  // light red
#     repl {background-color: #bb99ff;} // light purple
#     table tr > * { width: 50%; }
#     table tr > td { vertical-align: top; text-align: left; }
#     </style>
#     """

#     run_name = (
#         f"{row.task} - {row.strategy} - {row.msid} - {row.run}"
#         if row.task == "extract_figure_legends"
#         else f"{row.task} - {row.strategy} - {row.msid} - {row.figure_label} - {row.run}"
#     )
#     header = f"<h3>{run_name}</h3>"
#     section_fn = lambda title, content: f"<h4>{title}</h4><p>{content}</p>"
#     scores = section_fn("Scores", f"bleu1: {row.bleu1:.2f}, bleu2: {row.bleu2:.2f}, bleu3: {row.bleu3:.2f}, bleu4: {row.bleu4:.2f}, rouge1: {row.rouge1:.2f}, rouge2: {row.rouge2:.2f}, rougeL: {row.rougeL:.2f}")

#     comparison = f"""
# <table>
#     <thead>
#         <tr>
#             <th>Expected</th>
#             <th>Actual</th>
#         </tr>
#     </thead>
#     <tbody>
#         <tr>
#             <td>{html.escape(row.expected)}</td>
#             <td>{html.escape(row.actual)}</td>
#         </tr>
#     </tbody>
# </table>"""
#     diff = section_fn("Diff", inline_diff(row.expected, row.actual) + comparison)
#     input_text = section_fn("Input", row.input)

#     display(HTML(diff_css + header + scores + diff + input_text))

# def display_all(df):
#     print(f"Showing {len(df)} runs")
#     for row in df.itertuples():
#         display_diff(row)

# def show_runs(
#     df,
#     tasks=None,
#     strategies=None,
#     msids=None,
#     figure_labels=None,
#     runs=None,
#     score_range=None,
# ):
#     if tasks is not None:
#         df = df[df.task.isin(tasks)]
#     if strategies is not None:
#         df = df[df.strategy.isin(strategies)]
#     if msids is not None:
#         df = df[df.msid.isin(msids)]
#     if figure_labels is not None:
#         df = df[df.figure_label.isin(figure_labels)]
#     if runs is not None:
#         df = df[df.run.isin(runs)]
#     if score_range is not None:
#         min_score, max_score = score_range
#         df = df[(df[score] >= min_score) & (df[score] < max_score)]
#     return display_all(df)

In [ ]:
# for task_df in [legends, labels, titles, captions]:
#     if task_df.empty:
#         continue
#     print(f"Task: {task_df.task.iloc[0]}")
#     show_runs(task_df, score_range=(0., 0.949))